# 🦜🔗 RAG Fundamentals with LangChain & Groq

Welcome to **RAG (Retrieval Augmented Generation) Fundamentals**! This notebook teaches you how to build a RAG pipeline using LangChain and the **free Groq API**.

## What is RAG?
RAG combines **retrieval** (finding relevant documents) with **generation** (LLM response). Instead of relying only on the LLM's training data, we provide context from our own documents.

```
User Query → Retrieve Relevant Docs → Augment Prompt with Context → Generate Answer
```

## What You'll Learn:
1. **Document Loaders** - Load PDFs and other documents
2. **Text Splitters** - Chunk documents for processing
3. **Embeddings** - Convert text to vectors
4. **Vector Stores** - Store and search embeddings
5. **Retrievers** - Find relevant documents
6. **RAG Chains** - Build end-to-end question answering

## Dataset
We'll use **Landmark Judgments of the Supreme Court of India** PDF to build a legal Q&A system.

---

## Prerequisites
- Complete the LangChain Fundamentals notebook first
- Groq API key in your `.env` file

## 1. Setup & Installation

Install the required packages for RAG:

In [ ]:
# Install required packages (uncomment if needed)
# %pip install langchain langchain-groq langchain-huggingface langchain-community -q
# %pip install pypdf faiss-cpu sentence-transformers -q

In [1]:
# Import required libraries
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Verify API key
api_key = os.getenv("GROQ_API_KEY")
if not api_key:
    print("⚠️ GROQ_API_KEY not found! Set it in your .env file or uncomment below:")
    # os.environ["GROQ_API_KEY"] = "your-groq-api-key-here"
else:
    print("✅ GROQ_API_KEY loaded successfully!")

✅ GROQ_API_KEY loaded successfully!


In [2]:
from langchain_groq import ChatGroq
llm=ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0,
    max_tokens=1024
)
print(f" LLm initialied :{llm.model_name}")

 LLm initialied :llama-3.3-70b-versatile


## 2. Document Loaders

**Document Loaders** load data from various sources (PDFs, web pages, databases) into a standard `Document` format.

Each `Document` has:
- `page_content`: The text content
- `metadata`: Information about the source (page number, file name, etc.)

In [5]:
from langchain_community.document_loaders import PyPDFLoader
pdf_path="data/6.LANDMARK JUDGMENTS OF THE SUPREME COURT PLAIN.pdf"
loader=PyPDFLoader(pdf_path)
documents=loader.load()

print(f" Loaded {len(documents)} pages from pdf")

print(f"First page preview")
print(f"Content of first 500 chars:{documents[0].page_content[:500]}")
print(f"Meta data {documents[0].metadata} ")

 Loaded 11 pages from pdf
First page preview
Content of first 500 chars:Landmark  Judgments
• “......    When histories of nations are written 
and critiqued, there are judicial decisions at 
the forefront of liberty. Yet others have to be 
consigned to the archives, reflective of what 
was, but should never have been.”
D.Y Chandrachud J.
In Justice  K.S. Puttaswamy (retd.) Vs. Union 
of India and ors. , (2017) 10 SCC 1  
5
Meta data {'producer': 'PDFium', 'creator': 'PDFium', 'creationdate': 'D:20251121122306', 'source': 'data/6.LANDMARK JUDGMENTS OF THE SUPREME COURT PLAIN.pdf', 'total_pages': 11, 'page': 0, 'page_label': '1'} 


## 3. Text Splitters

LLMs have **context length limits**. We need to split documents into smaller **chunks** that:
- Fit within the model's context window
- Contain semantically meaningful content
- Have some overlap to maintain context between chunks

### Key Parameters:
- `chunk_size`: Maximum characters per chunk
- `chunk_overlap`: Characters shared between consecutive chunks

In [16]:
from langchain_text_splitters import RecursiveCharacterTextSplitter


# Create a text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,      # Max characters per chunk
    chunk_overlap=200,    # Overlap between chunks for context
    length_function=len,
    separators=["\n\n", "\n", ". ", " ", ""]  # Split priorities
)



In [18]:

chunks=text_splitter.split_documents(documents)

print(f" Split {len(documents)}  page into  {len(chunks)} chunks")


 Split 11  page into  11 chunks


In [19]:
print(chunks[5].page_content[:300])

2.Amendability of Fundamental Rights
Shankari Prasad v. Union of India , [1952] SCR 89 (1951): This 
case dealt with the amendability of Fundamental Rights 
(the First Amendment’s validity was challenged).
• The SC Court  held that the power conferred on 
Parliament by Art. 368 to amend is a very wi


In [20]:
print(chunks[5].metadata)

{'producer': 'PDFium', 'creator': 'PDFium', 'creationdate': 'D:20251121122306', 'source': 'data/6.LANDMARK JUDGMENTS OF THE SUPREME COURT PLAIN.pdf', 'total_pages': 11, 'page': 5, 'page_label': '6'}


## 4. Embeddings

**Embeddings** convert text into numerical vectors that capture semantic meaning. Similar texts have similar vectors.

```
"The court ruled in favor" → [0.23, -0.45, 0.67, ...]
"The judgment was positive" → [0.21, -0.43, 0.65, ...]  ← Similar vector!
```

We'll use **HuggingFace embeddings** (free, runs locally).

In [22]:
from langchain_huggingface import HuggingFaceEmbeddings
embedding=HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device':'cpu'}
)
sample_Text="The supereme court of India"
embedding_vector=embedding.embed_query(sample_Text)
print(len(embedding_vector))

384


In [23]:
print(embedding_vector[:5])

[-0.03839308023452759, 0.05466321483254433, -0.05524354428052902, -0.045149777084589005, -0.0801309123635292]


## 5. Vector Stores

**Vector Stores** are databases optimized for storing and searching embeddings. They enable fast **similarity search**.

Popular options:
- **FAISS** (Facebook AI) - Fast, in-memory, free
- **Chroma** - Easy to use, persistent storage
- **Pinecone** - Cloud-based, scalable

We'll use **FAISS** (free and fast).

In [24]:
from langchain_community.vectorstores import FAISS
vectorstore=FAISS.from_documents(
    documents=chunks,
    embedding=embedding
)
print(f"Vector store created with {len(chunks)} vectorstore")

Vector store created with 11 vectorstore


In [26]:
query="What is the Kesavandana Bharati case about?"
similar_docs=vectorstore.similarity_search(query,k=3)

print(f" query:{query}")
print(f" similar docs{similar_docs}")

 query:What is the Kesavandana Bharati case about?
 similar docs[Document(id='09866693-f03e-4d3e-affe-fef58a8775ad', metadata={'producer': 'PDFium', 'creator': 'PDFium', 'creationdate': 'D:20251121122306', 'source': 'data/6.LANDMARK JUDGMENTS OF THE SUPREME COURT PLAIN.pdf', 'total_pages': 11, 'page': 4, 'page_label': '5'}, page_content='LIFE AND LIBERTY: ADM Jabalpur v.  Shivakant\nShukla- OVERRULED \n• In Justice  K.S. Puttaswamy (retd.) Vs. Union of India and \nors. , (2017) 10 SCC 1  para 121 (Nine Judges), the apex \nCourt overruling the majority view  expressed in ADM \nJabalpur v. Shivkant Shukla (1976) 2 SCC 521,  held-\n• “The view taken by Justice Khanna must be accepted, \nand accepted in reverence for the strength of its thoughts \nand the courage of its convictions...”   \n• Sanjay Kishan Kaul, J. in his concurring judgment said: \n“...the ADM Jabalpur case which was an aberration in the \nconstitutional jurisprudence of our country and the \ndesirability of burying the ma

In [27]:
for i, doc in enumerate(similar_docs, 1):
    print(f"\n--- Document {i} (Page {doc.metadata.get('page', 'N/A')}) ---")
    print(doc.page_content[:300] + "...")


--- Document 1 (Page 4) ---
LIFE AND LIBERTY: ADM Jabalpur v.  Shivakant
Shukla- OVERRULED 
• In Justice  K.S. Puttaswamy (retd.) Vs. Union of India and 
ors. , (2017) 10 SCC 1  para 121 (Nine Judges), the apex 
Court overruling the majority view  expressed in ADM 
Jabalpur v. Shivkant Shukla (1976) 2 SCC 521,  held-
• “The vi...

--- Document 2 (Page 2) ---
Personal Liberty: Procedure Established by Law: 
Fair, Just and Reasonable
Maneka Gandhi v. Union of India (AIR 1978 SC 597)  (1978).
• The view expressed in A. K. Gopalan’s case was revisited in 
this case after about 28 years.
• The main issues were whether the right to go abroad is a 
part of the...

--- Document 3 (Page 7) ---
3. The Doctrine of Basic Structure 
His Holiness Kesavananda Bharati Sripadagalavaru v. State of 
Kerala,  AIR 1973 SC 1461 (decided by a Bench of 13 Judges)   
• The most celebrated  case in the history of Indian Constitutional 
law in which the apex Court dealt with the issue - whether the  
Parli...


## 6. Retrievers

A **Retriever** is an interface that returns relevant documents given a query. Vector stores can be converted to retrievers.

### Retriever Types:
- **Similarity** - Returns top k most similar documents
- **MMR (Maximum Marginal Relevance)** - Balances relevance with diversity
- **Self-Query** - LLM generates filter queries

In [28]:
retriever=vectorstore.as_retriever(
    search_type="similarity", 
    search_kwargs={"k":4}
)
query=" what are fundamental right?"
retrieved_document=retriever.invoke(query)


In [29]:

print(f"🔍 Query: '{query}'")
print(f"📄 Retrieved {len(retrieved_document)} documents")
for i, doc in enumerate(retrieved_document, 1):
    print(f"\n--- Doc {i} ---")
    print(doc.page_content[:200] + "...")

🔍 Query: ' what are fundamental right?'
📄 Retrieved 4 documents

--- Doc 1 ---
2.Amendability of Fundamental Rights
Shankari Prasad v. Union of India , [1952] SCR 89 (1951): This 
case dealt with the amendability of Fundamental Rights 
(the First Amendment’s validity was challen...

--- Doc 2 ---
Amendability of Fundamental Rights
The issues regarding power of the Parliament to amend Part 
III  of the Constitution was re-examined  in  I. C. Golak Nath
v. State of Punjab, AIR 1967 SC 1643 
• Th...

--- Doc 3 ---
The Doctrine of Basic Structure- Minerva Mills 
case, (1980) AIR 1980 SC 1789 
Validity of Constitution (42nd Amendment) Act, 1976 
,which inter alia provided for exclusion of judicial review of 
cons...

--- Doc 4 ---
The Doctrine of Basic Structure-
Application
Indira Nehru Gandhi v. Raj Narain 1975 SC 2299 
• The validity of 39th Constitution amendment 
enacted in 1975  which (Article 39-A ) sought to 
place  the...


## 7. RAG Chain - Putting It All Together

Now we combine everything into a **RAG Chain**:

```
Query → Retriever → Relevant Docs → Prompt + Context → LLM → Answer
```

We'll create a prompt that instructs the LLM to answer based ONLY on the provided context.

In [30]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
rag_prompt=ChatPromptTemplate.from_messages([
    ("system", """You are a helpful legal assistant specializing in Indian Supreme Court cases.
Answer the question based ONLY on the following context from landmark Supreme Court judgments.
If the answer is not found in the context, say "I cannot find this information in the provided documents."

Context:
{context}"""),
    ("human", "{question}")
])

print("✅ RAG prompt template created")

✅ RAG prompt template created


In [31]:
# Helper function to format retrieved documents
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Build the RAG chain using LCEL
rag_chain = (
    {
        "context": retriever | format_docs,  # Retrieve and format docs
        "question": RunnablePassthrough()     # Pass through the question
    }
    | rag_prompt     # Create the prompt
    | llm            # Generate response
    | StrOutputParser()  # Parse to string
)

print("✅ RAG chain created!")

✅ RAG chain created!


## 8. Test the RAG System

Let's ask questions about Indian Supreme Court landmark judgments!

In [32]:
question="What is the Kesavandana Bharati case and what is its significance?"
print(f" question :{question}")
print("\n\n")
answer=rag_chain.invoke(question)
print(f"ANswer :{answer}")

 question :What is the Kesavandana Bharati case and what is its significance?



ANswer :The Kesavananda Bharati case, formally known as His Holiness Kesavananda Bharati Sripadagalavaru v. State of Kerala (AIR 1973 SC 1461), is a landmark judgment in the history of Indian Constitutional law. 

In this case, the Supreme Court of India dealt with the issue of whether the Parliament can amend any part of the Constitution and what the limit to that power is. The Court, by a majority of 7-6, overruled the earlier proposition of law laid down in I.C. Golak Nath v. State of Punjab (AIR 1967 SC 1643) and held that:

1. Constitutional amendment is not "law" within the meaning of Article 13.
2. Although no part of the Constitution, including Part III comprising fundamental rights, is beyond the Parliament's amending power, 
3. The "basic structure of the Constitution" cannot be abrogated even by a constitutional amendment.

The significance of this case lies in the fact that it established the d

In [33]:
question="What did the supreme court say about the right to privacy?"
print(f" question :{question}")
print("\n\n")
answer=rag_chain.invoke(question)
print(f"ANswer :{answer}")

 question :What did the supreme court say about the right to privacy?



ANswer :I cannot find this information in the provided documents. The context only discusses the right to personal liberty under Article 21, the procedure established by law, and other related topics, but does not mention the right to privacy.


In [34]:
question="What did the supreme court say about the fundamental rights?"
print(f" question :{question}")
print("\n\n")
answer=rag_chain.invoke(question)
print(f"ANswer :{answer}")

 question :What did the supreme court say about the fundamental rights?



ANswer :According to the context, the Supreme Court made the following statements about the fundamental rights:

1. In the case of Shankari Prasad v. Union of India (1951), the Supreme Court held that the power conferred on Parliament by Article 368 to amend is a very wide power and includes the power to take away the fundamental rights guaranteed by Part III.

2. However, in the case of I. C. Golak Nath v. State of Punjab (1967), the Supreme Court overruled the earlier decision and held that the Legislature does not enjoy the power to amend Part III of the Constitution to take away or abridge fundamental rights.

In other words, the Supreme Court initially held that Parliament has the power to amend fundamental rights, but later changed its stance and held that fundamental rights cannot be amended by Parliament.


## 9. RAG with Source Attribution

Good RAG systems show **which documents** were used to generate the answer. Let's create a version that returns sources.

## 10. Streaming RAG Responses

For better user experience, stream the response token by token:

## 11. Save & Load Vector Store

Save the vector store to disk so you don't have to re-embed documents each time:

## 12. Interactive Q&A Function

Create a reusable function for asking questions:

## 🎯 Summary

In this notebook, you learned the **RAG Fundamentals**:

| Component | What You Learned |
|-----------|------------------|
| **Document Loaders** | Load PDFs with `PyPDFLoader` |
| **Text Splitters** | Chunk documents with `RecursiveCharacterTextSplitter` |
| **Embeddings** | Convert text to vectors with `HuggingFaceEmbeddings` |
| **Vector Stores** | Store and search with `FAISS` |
| **Retrievers** | Find relevant documents with similarity search |
| **RAG Chains** | Combine retrieval + generation with LCEL |
| **Source Attribution** | Return sources alongside answers |
| **Streaming** | Stream responses for better UX |
| **Persistence** | Save/load vector stores |

---

### RAG Pipeline Architecture:

```
┌─────────────┐    ┌─────────────┐    ┌─────────────┐
│  PDF/Docs   │───▶│   Chunks    │───▶│  Embeddings │
└─────────────┘    └─────────────┘    └─────────────┘
                                            │
                                            ▼
┌─────────────┐    ┌─────────────┐    ┌─────────────┐
│   Answer    │◀───│     LLM     │◀───│ Vector Store│
└─────────────┘    └─────────────┘    └─────────────┘
                          ▲                  │
                          │                  ▼
                   ┌─────────────┐    ┌─────────────┐
                   │   Prompt    │◀───│  Retriever  │◀── Query
                   └─────────────┘    └─────────────┘
```

### Next Steps:
- Add more document types (web pages, APIs)
- Experiment with different chunk sizes
- Try different embedding models
- Add conversation memory for multi-turn chat